# Tarea 1

**Envío del PR inicial:** viernes 21 de septiembre

**Aceptación del PR:** martes 25 de septiembre

## 1

Consideren el mapeo $F(x) = x^2-2$ definido en el intervalo $[-2,2]$. 

- ¿Qué podemos concluir después de muchos iterados para las condiciones iniciales $x_0=0$, $x_0=0.1$, $x_0=0.001$?
    
- ¿Qué propiedad hace que el interés en este mapeo sea el intervalo [-2,2]?


Para facilitar las cosas, conviene definir una función que simplifique el tomar los iterados de una función:

In [ ]:
"""
    iteraciones(f::Function, número_iteraciones::Int, x0)

Genera un array unidimensional de tamaño `1 + número_iteraciones` que en la entrada ``i + 1`` tiene al elemento proveniente de iterar `x0` ``ì`` veces con la función `f`. En el caso de ```i = 1``, el valor en el array es igual a la condición inicial usada.

# Argumentos

Requiere:

+ `f`, una función,
+ `número_iteraciones`, un entero que dicta el número de iteraciones a realizar, y,
+ `x0`, el punto inicial.

En particular, la condición inicial suministrada `x0` es convertida a su respectivo flotante antes de calcular los iterados; esto para facilitar el uso de arrays: es necesario que la función que se itera admita valores flotantes o estructuras con valores flotantes.

# Ejemplo
```julia-repl

julia> iteraciones(x->x^2, 5, 2)
6-element Array{Int64,1}:
          2
          4
         16
        256
      65536
 4294967296

julia> v = x -> [x[1] - x[2], x[1]*x[2]]; iteraciones(v, 2, [1, 3])
3-element Array{Array{Float64,1},1}:
 [1.0, 3.0]  
 [-2.0, 3.0] 
 [-5.0, -6.0]
```
"""
function iteraciones(f::Function, número_iteraciones::Int, x0)
    
    valor = float.(x0)
    
    iterados = [valor]
    
    for i in 1:número_iteraciones
        
        valor = f(valor)
        push!(iterados, valor)
    end
    
    return(iterados)
end

Así como eventualmente el valor de la iteración correspondiente:

In [ ]:
"""
    iterar(f::Function, número_iteraciones::Int, x0)

Devuelve el resultado de iterar `número_iteraciones` veces la función `f` con condición inicial `x0`.

# Argumentos

Requiere:

+ `f`, una función,
+ `número_iteraciones`, un entero que dicta el número de iteraciones a realizar, y,
+ `x0`, el punto inicial.

En particular, la condición inicial suministrada `x0` es convertida a su respectivo flotante antes de calcular los iterados; esto para facilitar el uso de arrays: es necesario que la función que se itera admita valores flotantes o estructuras con valores flotantes.

# Ejemplo
```julia-repl

julia> iterar(x->x^2, 5, 2)
4294967296

julia> v = x -> [x[1] - x[2], x[1]*x[2]]; iterar(v, 2, [1, 3])
2-element Array{Float64,1}:
 -5.0
 -6.0
```
"""
function iterar(f::Function, número_iteraciones::Int, x0)
    
    iterados = iteraciones(f, número_iteraciones, x0)
    valor = last(iterados)
    
    return(valor)
end

Para la función $ F(x) = x^2 - 2 $, consideremos el resultado de usar las condiciones iniciales dadas iterando $F$ 1000 veces:

In [ ]:
F(x) = x^2 - 2

x0 = 0
número_iteraciones = 1000

iteraciones_F_1 = iteraciones(F, número_iteraciones, x0)

Entonces, $x_0 = 0$ es un punto eventualmente periódico, más aún, la órbita a la que converge es de periodo uno, lo que indica que la iteración converge eventualmente a un punto fijo: $x = 2$.

In [ ]:
x0 = 0.1
iteraciones_F_2 = iteraciones(F, número_iteraciones, x0)

No se observa ningún tipo de patrón evidente, aunque una mejor forma de investigar esto es realizando una gráfica:

In [ ]:
using Plots
using LaTeXStrings
pyplot()

scatter(iteraciones_F_2, key = false, xlabel = L"n", ylabel = L"x_n", title = LaTeXString("Primeras 1000 iteraciones de \$F\$ con \$x_0\$ = $x0"))

No se observa periodicidad, aunque se nota que todos los valores resultantes están en el intervalo [-2, 2].

In [ ]:
x0 = 0.001
iteraciones_F_3 = iteraciones(F, número_iteraciones, x0)

No se observa ningún tipo de patrón evidente, aunque una mejor forma de investigar esto es realizando una gráfica:

In [ ]:
scatter(iteraciones_F_3, key = false, xlabel = L"n", ylabel = L"x_n", title = LaTeXString("Primeras 1000 iteraciones de \$F\$ con \$x_0\$ = $x0"))

No se observa periodicidad, aunque se nota que todos los valores resultantes están en el intervalo [-2, 2].

Como se pudo observar en los ejemplos, existió un punto que fue eventualmente periódico y otros dos que aparentemente no tienen ningún patrón que sea evidente. Sin embargo, en todos los casos el valor de las iteraciones permanece dentro del intervalo [-2, 2], esto es por lo siguiente:

Sea $ x \in [-2, 2]$, entonces, $x^2 \in [0, 4] \implies F(x) = x^2 -2 \in [-2, 2] $. 

Esto quiere decir que todas las condiciones iniciales tomadas en el intervalo [-2, 2] son mapeadas al mismo intervalo con la función $F$. Esto, considerando los ejemplos anteriores, nos indica que hay mapeos que no son periódicos o eventualmente periódicos pero acotados.

## 2

Escriban una función cuyos argumentos son la función `F` del mapeo, `x0`, la condición inicial, y `n` el número de iterados del mapeo (un número entero), que genere los `n` iterados del mapeo y haga el análisis gráfico descrito del mapeo. Ilustren el funcionamiento con $F(x)=\sqrt{x}$.

¿Qué pueden concluir de iterar *muchas* veces una condición inicial?

Una forma sencilla de implementar lo solicitado es generalizando el código del notebook de clase:

In [ ]:
"""
    análisis_gráfico(f::Function, número_iteraciones::Int, x0::Array{N, 1}, a::Real, b::Real, paso::Real, c::Real, d::Real; title::String = "", ylabel::String = "\$f(x)\$", colores::Array) where N <: Real

Genera un análisis gráfico de los iterados de la función `f` por el `número_iteraciones` dado con condiciones iniciales iguales a las entradas del vector `x0`. La ventana de graficación usada está dada por `` [a, b]\\times[c, d] ``. Para graficar la función, se usan puntos de muestreo separados por `paso` en el intervalo `` [a,b] ``. El título y el título del eje para los iterados se pueden cambiar usando los mismos argumentos que en `Plots.jl`. Los colores de las líneas para los iterados se deben suministrar como un array de colores al keyword argument `colores`.

# Argumentos

Requiere:

+ `f`, una función,
+ `número_iteraciones`, un entero que dicta el número de iteraciones a realizar,
+ `x0`, el array de puntos iniciales usado,
+ `a`, `b`, `c` y `d`, los parámetros usados para establecer la ventana de visualización, y,
+ `colores`, el array de colores para las líneas de los iterados.

En particular, las condiciones iniciales suministradas (las componentes de `x0`) son convertidas a sus respectivos flotantes antes de calcular los iterados; esto para facilitar el uso de arrays: es necesario que la función que se itera admita valores flotantes o estructuras con valores flotantes.

Opcionalmente se pueden especificar:

+ `title`, el título de la gráfica mediante un string, y,
+ `ylabel`, el título del eje y mediante un string.

En particular, `title` y `ylabel` son interpretados como strings con ecuaciones escritas en  `` \\LaTeX `` mediante el comando `LaTeXString`. Esto quiere decir que el texto no es interpretado dentro del entorno matemático de  `` \\LaTeX `` sin incluirlo entre pares de  `` \\ \$ `` (signos de dinero escapados).

# Ejemplo

Para visualizar los primeros cinco iterados de la función `` x \\mapsto \\sqrt{x} `` en la ventana `` [0, 1] \\times [0,1] `` con la función graficada con una resolución de paso igual a 0.01 con condición inicial 0.5 se puede usar el comando:
```julia-repl

julia> análisis_gráfico(x->sqrt(x), 5, [0.5], 0, 1, 0.01, 0, 1, colores = [:green])
```

Para hacer lo mismo, pero cambiando el título a "Ejemplo", el título del eje y a "Eje y" y agregando los iterados correspondientes a la condición inicial igual a 0.7 en color rojo se puede usar:
```julia-repl

julia> análisis_gráfico(x->sqrt(x), 5, [0.5, 0.7], 0, 1, 0.01, 0, 1, title = "Ejemplo", ylabel = "Eje y", colores = [:green, :red])
```
"""
function análisis_gráfico(f::Function, número_iteraciones::Int, x0::Array{N, 1}, a::Real, b::Real, paso::Real, c::Real, d::Real; title::String = "", ylabel::String = "\$f(x)\$", colors::Array) where N <: Real

    #Convirtiendo los elementos de los puntos muestra a flotantes:
    a = float(a)
    b = float(b)
    paso = float(paso)

    #Generando la gráfica de la función:
    rango_x = a:paso:b
    
    título = LaTeXString(title)
    leyenda_y = LaTeXString(ylabel)
    
    gráfica = plot(rango_x, f, 
        xaxis = (L"x", (a, b)),
        yaxis = (leyenda_y, (c, d)),
        legend = false, 
        title = título, 
        grid = false,
        label = "Función iterada")
    
    #Y de la identidad:
    plot!(gráfica, x -> x, color = :red, label = L"Id")
    
    #El eje x:
    plot!(gráfica, x -> 0, color = :orange)
    
    #Generando las gráficas de los iterados
    número_condiciones_iniciales = length(x0)
    
    for i in 1:número_condiciones_iniciales
        
        valor_inicial = float(x0[i])
        color_i = colors[i]
        
        #Cargando los iterados:
        iterados = iteraciones(f, número_iteraciones, valor_inicial)

        #Gráfica de los iterados iniciales:
        x_0 = iterados[1]
        x_1 = iterados[2]

        x_iniciales = [x_0, x_0, x_1]
        y_iniciales = [0, x_1, x_1]

        plot!(gráfica, x_iniciales, y_iniciales, 
            line=(color_i, :path, 2.0, :dot), 
            marker=(:circle, 0.5), 
            label = "x0 = $x_0")

        #Gráfica del resto de los iterados:
        for i in 2:número_iteraciones

            x_actual = iterados[i]
            x_siguiente =  iterados[i+1]

            x = [x_actual, x_actual, x_siguiente]
            y = [x_actual, x_siguiente, x_siguiente]

            plot!(gráfica, x, y, 
                line=(color_i, :path, 2.0, :dot), 
                marker=(:circle, 0.5),
                label = "")
        end
    end
    
    return(gráfica)
end

Para realizar el análisis de la función $F_2(x) = \sqrt{x}$ en varias condiciones iniciales se puede utilizar el macro `@manipulate` de la paquetería `Interact.jl`:

In [ ]:
using Interact

In [ ]:
F₂(x) = sqrt(x)
número_iteraciones = 5
a = 0; b = 5; paso = 0.01; c = 0; d = 3;
título = "Análisis para \$ F_2(x) = \\sqrt{x} \$"

@manipulate for x0 in 0:0.1:4

    análisis_gráfico(F₂, número_iteraciones, [x0], a, b, paso, c, d, title = título, colors = [:black])
end

Como se puede observar, se tienen dos puntos fijos: uno en $x_0 = \sqrt{x_0} = 0$ y otro en $x_1 = \sqrt{x_1} = 1$. Más aún, para el muestreo de puntos usado distintos de $x_0$, se observa que $x_1$ tiende a "atraer" las trayectorias de otros puntos. Esto se observa gráficamente debido a que las trayectorias generadas se acercan a este punto y el cambio generado por la iteración de la función es menor en la región en torno a $x_1$ en comparación a regiones "lejanas" a este punto. Esto se puede observar analíticamente observando que: $0 < x < 1 \implies 0 < x < \sqrt{x} < 1 $ y $1 < x \implies  1 < \sqrt{x} < x < 1$.

## 3

- Usen el análisis gráfico para encontrar los puntos de periodo dos del mapeo $F(x) = x^2 -1.1$. Den explícitamente los puntos fijos.
    
- Usen el método de Newton, usando iterados iniciales  sagazmente seleccionados, para obtener los puntos fijos.

Los puntos fijos de una función $f(x)$ deben satisfacer:

$$ x = f(x) \implies f(x) - x = 0. $$

Por lo que, para la función $F_3(x) = x^2 - 1.1$: $ x^2 -x -1.1 = 0 \implies x = \frac{1}{2} \pm \frac{\sqrt{1 + 4(1.1)}}{2} = \frac{1 \pm \sqrt{5.4} }{2} $. Esto significa que los dos puntos fijos de la función $F_3$ son: 

+ $x_0 = \frac{1 + \sqrt{5.4} }{2} \approx 1.6619$, y,
+ $x_1 = \frac{1 - \sqrt{5.4} }{2} \approx -0.6619$.

Realizando el análisis gráfico de la función $F_3(x) = x^2 - 1.1$ con los siguientes parámetros:

In [ ]:
F₃(x) = x^2 - 1.1
número_iteraciones = 100
a = -3; b = 3; paso = 0.05; c = -1.1; d = 10;
título = "Análisis para \$ F_3(x) = x^2 - 1.1 \$"

@manipulate for x0 in -2:0.01:2

    gráfica = análisis_gráfico(F₃, número_iteraciones, [x0], a, b, paso, c, d, title = título, colors = [:green])
end

Usando estos parámetros se encuentra que $ y_0 \approx -1.09 $ y $ y_1 \approx 0.09 $ son dos de los puntos con periodo dos distintos de los puntos de periodo uno $x_0$ y $x_1$. (Ya que un punto fijo por definición es de periodo arbitrario tmabién.) Adicionalmente $ x \approx -1.48 $ y $ x \approx 1.09 $ son puntos que aparentan ser eventualmente de periodo 2. 

Que solo existan cuatro puntos de periodo dos en todo el dominio ($x_0$, $x_1$, $y_0$ y $y_1$), se puede entender usando el teorema fundamental del álgebra de la siguiente forma:

Un punto $x^*$ de periodo $n$ de la función $G$ debe de satisfacer que:

$$ G^n(x^*) = x^ * \iff G^n(x^*) - x^* = 0, $$

dónde $G^n$ es la función resultante de iterar $ n $ veces la función $G$.

Esto implica que los puntos de periodo 2 de $ F_3 $ deben de satisfacer:

$$ F_3^2(y) - x^* = F_3(F_3(y)) - y = F_3(y^2 - 1.1) - x^* = (y^2 -1.1)^2 - 1.1 - x^* = (y)^4 - 2.2 y^2 - y - + (1.1)^2 - 1.1 $$

Como esto es un polinomio de grado 4, cuanto más, existen cuatro puntos (complejos) que son raíces de esta ecuación- 

Para conocer los puntos que satisfacen esta ecuación directamente desde Julia, se puede usar el paquete `SymPy.jl`. Para esto, conviene definir una función que de la n-ésima iteración "simbólica" de una función dada:

In [ ]:
"""
    función_iterar(f::Function, número_iteraciones::Int)

Devuelve la función "simbólica" proveniente de iterar `f` el `número_iteraciones` dado.

# Argumentos

Requiere:

+ `f`, una función, y,
+ `número_iteraciones`, un entero que dicta el número de iteraciones a realizar.

# Ejemplo
```julia-repl

julia> f = x -> x^2 - 1; g = función_iterar(f, 2); g(1)
-1

julia> f = x -> x^2 - 1; g = función_iterar(f, 2); g(0)
0
```
"""
function función_iterar(f::Function, número_iteraciones::Int)
    
    if número_iteraciones == 1
        
        return(f)
        
    else
        
        return(x -> f(función_iterar(f, número_iteraciones - 1)(x)))
            
    end
end

La función cuyas raíces son los puntos de periodo $k$ de $F_3$ está dada por:

In [ ]:
puntos_fijos_F₃(k, x) = función_iterar(F₃, k)(x) - x
puntos_fijos_F₃(k) = x -> puntos_fijos_F₃(k, x)

Los puntos de periodo uno son:

In [ ]:
using SymPy

y = Sym("y")
puntos_fijos = solve(puntos_fijos_F₃(1)(y))

Y los puntos de periodo dos son:

In [ ]:
puntos_periodo_2 = solve(puntos_fijos_F₃(2)(y)) |> float

Como se puede observar, los puntos estudiados gráficamente son obtenidos numéricamente a partir de métodos simbólicos.

Obsérvese que esto no quiere decir que los únicos puntos que generan trayectorias eventualmente de periodo dos sean estos, sino que si algún punto ($ x \approx -1.48, 1.09 $, por ejemplo) contiene a alguno de los cuatro puntos mencionados como algún iterado, la trayectoria "final" de estos es idéntica a la "total" de los cuatro puntos mencionados.

Finalmente, para apreciar que en efecto estos puntos son los de periodo dos:

In [ ]:
using Colors

In [ ]:
número_iteraciones = 100
x0 = puntos_periodo_2
a = -2; b = 2; paso = 0.05; c = -1.1; d = 3;
título = "Análisis de los puntos de periodo dos para \$ F_3(x) \$"
título_y = " \$ F_3(x) \$"
colores = [colormap("Greens", 8)[2*i] for i in 1:4]

gráfica_puntos_periodo_2 = análisis_gráfico(F₃, número_iteraciones, x0, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Y los puntos eventualmente periódicos encontrados son:

In [ ]:
número_iteraciones = 100
x0 = x0 = [-1.48, 1.09]
a = -2; b = 2; paso = 0.05; c = -1.1; d = 3;
título = "Análisis de los puntos eventualmente de periodo dos, \$ F_3(x) \$"
título_y = " \$ F_3(x) \$"
colores = [colormap("Greens", 8)[4*i] for i in 1:2]

gráfica_puntos_eventualmente_periodo_2 = análisis_gráfico(F₃, número_iteraciones, x0, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Una idea para encontrar otros puntos que son eventualmente periódicos es ver las raíces de la función cuando esta es igualada a algún punto de periodo conocido. Descartando los puntos que ya se saben forman parte de la órbita o son otros puntos eventualmente periódicos permiten tener una lista de puntos que poseen órbitas eventualmente periódicas. 

Para poder encontrar los puntos de periodo 2 usando herramientas desarrolladas en el curso, conviene cargar los siguientes [módulos](https://docs.julialang.org/en/v1/manual/modules/): 

In [ ]:
push!(LOAD_PATH, "../Código")

using Newton
using DifAutom

Definiendo la derivada de la función para calcular los puntos fijos usando duales:

In [ ]:
puntos_fijos_F₃_prime(k) = x -> derivada_dual(puntos_fijos_F₃(k), x)

Para poder determinar algunas condiciones iniciales de interés, conviene graficar la función cuyas raíces son los puntos de periodo 1:

In [ ]:
plot(-2:0.01:2, puntos_fijos_F₃(1), key = false, title = "Gráfica de la función para determinar puntos fijos", xlabel = L"x", ylabel = L"F_3(x) - x")

Como se puede observar, tomando $x_0 = \pm 2$, probablemente se tienen los puntos fijos de $F_3$ como resultado de aplicar el método de Newton a la función anterior.

Calculando los puntos fijos usando el método de Newton en la función adecuada (`puntos_fijos_F₃(k)` con $k = 1$):

In [ ]:
k = 1

@show x0 = 2
@show newton(puntos_fijos_F₃(k), puntos_fijos_F₃_prime(k), x0)

@show x0 = -2
@show newton(puntos_fijos_F₃(k), puntos_fijos_F₃_prime(k), x0);

Se obseva un acuerdo "suficiente" (un ligero desacuerdo en la última cifra decimal en uno de los puntos) con los puntos fijos calculados numéricamente por `SymPy.jl` y el cálculo numérico con herramientas desarrolladas en el curso. Más aún, como:

In [ ]:
@show (1 - sqrt(5.4))/2
@show (1 + sqrt(5.4))/2;

Se observa un acuerdo en general con los resultados obtenidos analítica y numéricamente.

## 4

Hagan un análisis completo de las posibles órbitas de los siguientes mapeos, usando distintos colores para distintas órbitas. Interesa entre otras cosas saber cuántos puntos fijos tiene y cómo es la dinámica cerca de ésos puntos.

- $F_1(x) = \frac{1}{3x}$.

- $F_2(x) = -\frac{x^5}{5}$.

- $F_3(x) = \frac{x}{2}-2$.

#### 1.- $F_1(x) = \frac{1}{3x}$

Definiendo $F_1(x)$:

In [ ]:
F₁(x) = 1/(3*x)

Para darse una idea general de lo que está ocurriendo, se puede hacer algo similar que en el ejercicio anterior:

In [ ]:
número_iteraciones = 10
a = -5; b = 5; paso = 0.01; c = -10; d = 10;
título = "Análisis para \$ F_1(x) = \\frac{1}{3x} \$"

@manipulate for x0 in -5:0.1:5

    análisis_gráfico(F₁, número_iteraciones, [x0], a, b, paso, c, d, title = título, colors = [:green])
end

Aparentemente todos los puntos de muestreo, salvo cero, son puntos de periodo dos. Destacan como puntos fijos los esperados: la intersección de la hipérbola con la identidad. Más aún, se observa que la órbita generada por los puntos de muestreo está confinada a la rama de la hipérbola a la que pertenece el punto inicial. Todo esto se puede demostrar formalmente o calcular de la siguiente manera:

+ **Todos los puntos en el dominio son puntos de periodo dos:**

Se tiene que si $ x \in \mathbb{R}-\{0\}$,  $F_2^2(x) = F_2(F_2(x)) = F_2(\frac{1}{3x}) = \frac{1}{3\left(\frac{1}{3x}\right)} = \frac{1}{\left(\frac{1}{x}\right)} = x$.

+ **Los puntos fijos son la intersección de la hipérbola con la identidad:**

Los puntos fijos satisfacen que: $ x = F_2(x)$. De esto, se tiene que:

$$ x \in \mathbb{R} - \{0\} \implies F_2(x) = \frac{1}{3x} = x \iff 3x^2 - 1 = 0 \iff x = \pm \frac{1}{\sqrt{3}}$$

+ **Las órbitas generadas por los puntos del dominio están contenidas en la rama de hipérbola a la que pertenece el punto inicial:**

Supóngase s.p.g. que $ x > 0 $, entonces $F_1(x) = \frac{1}{3x} > 0$. Como un punto en el dominio es siempre de periodo 2, se tiene el resultado para la rama derecha. El resultado para la otra rama se puede demostrar análogamente.

Para estudiar la dinámica de los puntos cercanos a los puntos fijos, se puede realizar lo siguiente:

In [ ]:
#Considerando un ensamble de puntos en torno a:
x_1_a = -1/sqrt(3)
ensamble_1_a = (x_1_a - 0.5):0.1:(x_1_a + 0.5) |> collect

número_iteraciones = 10
a = -2; b = 0.1; paso = 0.001; c = -2; d = 0;
título = "Análisis de los puntos fijos, \$F_1(x)\$"
colores = [colormap("Greens", 22)[i] for i in 11:22]

análisis_F₁_a = análisis_gráfico(F₁, número_iteraciones, ensamble_1_a, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Se observa que los puntos de lado izquierdo al punto fijo generan órbitas más "cerradas" que los simétricos respecto al punto fijo correspondientes del lado derecho. Las órbitas son más cerradas al acercarse los puntos iniciales al punto fijo. 

In [ ]:
#Considerando un ensamble de puntos en torno a:
x_1_b = 1/sqrt(3)
ensamble_1_b = (x_1_b - 0.5):0.1:(x_1_b + 0.5) |> collect

número_iteraciones = 10
a = 0.1; b = 2; paso = 0.001; c = 0; d = 2;
título = "Análisis de los puntos fijos, \$F_1(x)\$"
colores = [colormap("Greens", 22)[i] for i in 11:22]

análisis_F₁_b = análisis_gráfico(F₁, número_iteraciones, ensamble_1_b, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Se observa algo similar que en el otro punto fijo, salvo que las órbitas más cerradas se encuentran a la derecha del punto cuando se comparan puntos simétricos respecto al punto fijo. El origen se comporta como un "repulsor"; esto se explica por la indeterminación de la derivada de la función $F_1(x)$ en el origen.

#### 2.- $F_2(x) = -\frac{x^5}{5}$

Definiendo $F_2(x)$:

In [ ]:
F₂(x) = -x^5 / 5

Para darse una idea general de lo que está ocurriendo, se puede hacer algo similar que en el ejercicio anterior:

In [ ]:
número_iteraciones = 10
a = -2; b = 2; paso = 0.01; c = -5; d = 5;
título = "Análisis para \$ F_2(x) = -\\frac{x^5}{5} \$"

@manipulate for x0 in -2:0.01:2

    análisis_gráfico(F₂, número_iteraciones, [x0], a, b, paso, c, d, title = título, colors = [:green])
end

Se observa que para los puntos de muestreo fuera del intervalo [-1.4, 1.4] las órbitas divergen, mientras que para los puntos al interior del intervalo, las órbitas convergen a cero. El único punto fijo identificable se encuentra en el origen. No se observan puntos de periodo dos distintos del origen dentro del conjunto de puntos muestra, sin embargo, existen puntos que no se visualizan en la muestra anteriror. Esto se puede entender de la siguiente forma:

+ **Puntos de periodo dos:**

Se tiene que $F_2^2(x) = F_2(F_2(x)) = -\frac{1}{5} \left( -\frac{x^5}{5} \right)^5 = \frac{x^{25}}{5^6} $. Como los puntos de periodo dos deben satisfacer que $F_2^2(x) = x = \frac{x^{25}}{5^6}$, se tiene que $ x^{25} - 5^6 x = x(x^{24} - 5^6) = 0$.

Usando productos notables se tiene que:

\begin{align}
    0 &= x(x^{24} - 5^6) \\
      &= x(x^{12} + 5^3)(x^{6} + 5^\frac{3}{2})(x^{3} + 5^\frac{3}{4})(x^{3} - 5^\frac{3}{4}) \\
      &= x(x^{12} + 5^3)(x^{6} + 5^\frac{3}{2})(x + 5^\frac{1}{4})(x^2 - 5^\frac{1}{4}x + 5^\frac{1}{2})(x - 5^\frac{1}{4})(x^2 + 5^\frac{1}{4}x + 5^\frac{1}{2})
\end{align}

De dónde, dado que $ \pm 5^\frac{1}{2} - 4 \cdot 5^\frac{1}{2} = (\pm 1 - 4)5^\frac{1}{2} < 0$ (un análisis del discriminante de los factores cuadráticos), se tiene que los únicos puntos reales de periodo dos son: $x = 0, \pm \sqrt[4]{5}$. Esto explica el por qué no se observaron en la visualización los últimos dos puntos: debido a que son irracionales.

+ **Puntos fijos**

Dado que los puntos fijos deben de satisfacer la ecuación $F_2(x) = x = - \frac{x^5}{5}$, se tiene que $x^5 + 5 x = x (x^4 + 5)$, por lo que el único punto fijo real es $x = 0$

+ **Cotas para el cambio de comportamiento de las órbitas que convergen a cero**

Para saber en qué punto las órbitas pasan de ser convergentes a divergentes, se puede hacer lo siguiente:

Una órbita convergente a cero debe de satisfacer que: $ |F_2(x)| = \frac{|x|^5}{5} < |x|$. De dónde:

$$ |x|^5 - 5 |x| = |x|(x^4 - 5) < 0 \iff x^4 < 5  \implies |x| < \sqrt[4]{5}.$$

Por lo que toda órbita convergente a cero debe tener su punto inicial en el intervalo $(-\sqrt[4]{5}, \sqrt[4]{5}) \approx (-1.49, 1.49)$. Por un argumento similar, toda órbita divergente de cero debe tener su punto inicial en el exterior de dicho intervalo. Dada la generalidad de este argumento, es posible asegurar que toda trayectoria con punto inicial en dicho intervalo converge a cero y toda trayectoria afuera de dicho intervalo no converge (es divergente o periódica fuera del origen).

Para estudiar la dinámica de los puntos cercanos al origen, se puede realizar lo siguiente:

In [ ]:
#Considerando un ensamble de puntos en torno a:
x_2_a = 0
ensamble_2_a = (x_2_a - 1.6):0.1:(x_2_a + 1.6) |> collect

número_iteraciones = 10
a = -3; b = 3; paso = 0.001; c = -2; d = 2;
título = "Análisis de los puntos fijos, \$F_2(x)\$"
colores = [colormap("Greens", 66)[i] for i in 33:66]

análisis_F₂_a = análisis_gráfico(F₂, número_iteraciones, ensamble_2_a, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Como se puede observar, los puntos al interior del intervalo señalado tienen órbitas convergentes, pero cada vez más "abiertas" conforme se acercan a los extremos del mismo. En el momento en el que llegan al extremo, se vuelven periódicas. Fuera de este intervalo, las órbitas divergen.

#### 3.- $F_3(x) = \frac{x}{2} - 2$

Definiendo $F_3(x)$:

In [ ]:
F₃(x) = x/2 - 2

Para darse una idea general de lo que está ocurriendo, se puede hacer algo similar que en ejercicios anteriores:

In [ ]:
número_iteraciones = 10
a = -9; b = 1; paso = 0.01; c = -10; d = 0;
título = "Análisis para \$ F_3(x) = \\frac{x}{2} - 2 \$"

@manipulate for x0 in -9:0.01:1

    análisis_gráfico(F₃, número_iteraciones, [x0], a, b, paso, c, d, title = título, colors = [:green])
end

Se observa que todos los puntos muestra generan órbitas convergentes a $x = -4$, el único punto fijo encontrado.

+ **Punto fijos**

Dado que los puntos fijos deben de satisfacer la ecuación $F_3(x) = x = \frac{x}{2} - 2$, se tiene que el único punto fijo es $x = -4$.

+ **Puntos de periodo dos:**

Se tiene que $F_3^2(x) = F_3(F_3(x)) = -\frac{1}{2} \left( \frac{x}{2} - 2 \right) - 2 = \frac{x}{4} - 3 $. Como los puntos de periodo dos deben satisfacer que $F_3^2(x) = x = \frac{x}{4} - 3$, se tiene que $ x = -4$. (No hay puntos de orden dos distintos al punto fijo.)

Para estudiar la dinámica de los puntos cercanos al origen, se puede realizar lo siguiente:

In [ ]:
#Considerando un ensamble de puntos en torno a:
x_3_a = -4
ensamble_3_a = (x_3_a - 1):0.1:(x_3_a + 1) |> collect

número_iteraciones = 10
a = -5; b = -3; paso = 0.001; c = -5; d = 0;
título = "Análisis de los puntos fijos, \$F_3(x)\$"
colores = [colormap("Greens", 42)[i] for i in 21:42]

análisis_F₃_a = análisis_gráfico(F₃, número_iteraciones, ensamble_3_a, a, b, paso, c, d, title = título, ylabel = título_y, colors = colores)

Como se puede observar, todas las órbitas generadas por los puntos muestra son convergentes al punto fijo.

## 5

- Para el mapeo mapeo $F(x) = x^2-2$ en el intervalo $[-2,2]$ (ejercicio 1), consideren un punto al azar en el intervalo $[-2,2]$ y construyan una órbita muy larga, por ejemplo, de $20\,000$ iterados. Obtengan el histograma de frecuencia (normalizado) de visita de los puntos de la órbita. ¿Qué conclusión podemos sacar de este histograma?

- \Repitan el inciso anterior para el mapeo: $G(x) = 2x \mod 1$ en el intervalo [0,1].

(NOTA: En cuanto al histograma, pueden construir su propia función que haga el histograma, o buscar en las entrañas de la documentación.)

## 6

Analicen los mapeos

- F(x) = x^2,

- F(x) = x^3,

- F(x) = -x^3,

que se satisface $F(0) = F'(0) = 0$. ¿Qué podemos decir de la convergencia?

## 7

Analicen los mapeos

- F(x) = x,

- F(x) = -x,

- F(x) = x-x^2,

que se satisface $F(0) = 0$ y $|F'(0)| = 1$. (Este tipo de puntos son *neutralmente* atractivos o repulsivos.) ¿Qué podemos decir de la convergencia?

## 8

En el ejercicio 3, implementaron el análisis gráfico (y el método de Newton) para encontrar los puntos de periodo dos del mapeo $F(x) = x^2 -1.1$. Usen el módulo de duales para mostrar la estabilidad de esos puntos.